# How to build a simple Agent LLM App with LangGraph
* Very basic tool-using Agent LLM App with memory.

Language models can't do anything on their own; they can only create text based on what you ask them. However, LangChain allows people to build agents—think of these as smart systems or helpers— that use language models to think and decide what to do next.

Here’s how it works:
1. **Use the Language Model as a Brain**: The agent uses the language model to figure out which actions it should take, based on the information it has or what you ask it to do.
2. **Taking Action**: After deciding, the agent then goes ahead and does those actions.
3. **Learning and Adjusting**: Once the actions are done, the results can be given back to the language model. This helps the model check if everything is complete or if it needs to do something else.

So, essentially, LangChain helps turn a language model from just a tool for writing and answering into a system that can act and react, almost like a very simple robot brain.

Here we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it calling the search tool, and have conversations with it.

## Concepts included
* [LangGraph](https://python.langchain.com/v0.2/docs/concepts/#langgraph).

LangGraph is a library created by the LangChain team for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo')

### **Define tools**

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke('who won the 2024 Eurocup?')

[{'url': 'https://www.nbcnewyork.com/news/sports/soccer/spain-england-euro-final-score-result/5595340/?os=vb_&ref=app',
  'content': 'Spain is back on top.. La Furia Roja on Sunday claimed the Euro 2024 trophy after topping England 2-1 in the final. It marked consecutive Euro final losses for the Three Lions after they fell to ...'},
 {'url': 'https://www.nbcnews.com/news/sports/spain-england-euro-final-recap-rcna161803',
  'content': 'July 14, 2024, 2:04 PM PDT. By Greg Rosenstein. Champions of Europe once again. Spain defeated England 2-1 on Sunday in Berlin to claim the European Championship. The result was its fourth ...'}]

In [6]:
#define list of tools for the agent
tools=[search]

* **In order to enable this model to do tool calling we use ,bind_tools** to give the language model knowledge of these tools:

In [8]:
llm_with_tools = llm.bind_tools(tools)

### **Create the Agent**

## Create the agent
* We will be using LangGraph to construct the agent. 
* **Note that below we are passing in the origina chat model, not the llm_with_tools we built later**. That is because create_tool_calling_executor will call .bind_tools for us under the hood.

In [11]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm,tools)

### **Run the agent**


* lets first try it with .invoke()

In [15]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({
    'messages': [HumanMessage(content='where is the soccer Eurocup 2024 played')]
})

response['messages']

[HumanMessage(content='where is the soccer Eurocup 2024 played', additional_kwargs={}, response_metadata={}, id='9ebdd1ec-3d10-4e19-a625-2c167db27193'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mPq5IgUBOydNKR1Mu2nZqbBJ', 'function': {'arguments': '{"query":"Soccer Eurocup 2024 location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 91, 'total_tokens': 116, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4f5dc838-a249-47af-a43c-7a857558741d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Soccer Eurocup 2024 location'}, 'id': 'call_mPq5IgUBOydNKR1Mu2nZqbBJ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 25, 'total_tokens': 116}),
 ToolMessage(content='[{"url": "https

In [16]:
for chunk in agent_executor.stream(
    {'messages': [HumanMessage(content='when and where was the 2024 Eurocup final match?')]}
):
    print(chunk)
    print('--------')

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_X6Wf5qRMxN1RWjZV2IxFUtry', 'function': {'arguments': '{"query":"2024 Eurocup final match date and location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 94, 'total_tokens': 120, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7b723055-b53c-4a58-88ed-4f9ebcf3413f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 Eurocup final match date and location'}, 'id': 'call_X6Wf5qRMxN1RWjZV2IxFUtry', 'type': 'tool_call'}], usage_metadata={'input_tokens': 94, 'output_tokens': 26, 'total_tokens': 120})]}}
--------
{'tools': {'messages': [ToolMessage(content='[{"url": "https://en.wikipedia.org/wiki/UEFA_Euro_2024_Final", "content": "The UEFA

### **Adding memory**

* Adding memory in LangGraph is very similar to what we did with LangChain

In [18]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
agent_executor = create_react_agent(llm,tools,checkpointer=memory)
config = {'configurable': {'thread_id': '001'}} 

In [22]:
for chunk in agent_executor.stream(
    {'messages': HumanMessage(content='Who won the 2024 Eurocup?')}, config
):
    print(chunk)
    print('---------------')

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VUhMmDEjZfDYWaoYeykZ527u', 'function': {'arguments': '{"query":"2024 Eurocup winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 90, 'total_tokens': 112, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-70c04385-4bce-4a88-95f9-b785ed3febcd-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 Eurocup winner'}, 'id': 'call_VUhMmDEjZfDYWaoYeykZ527u', 'type': 'tool_call'}], usage_metadata={'input_tokens': 90, 'output_tokens': 22, 'total_tokens': 112})]}}
---------------
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.usatoday.com/story/sports/soccer/europe/2024/07/14/england-vs-spain-euro-2024-odds-prediction-highli

In [23]:
for chunk in agent_executor.stream(
    {'messages': HumanMessage(content='Who where top rated players?')}, config
):
    print(chunk)
    print('---------------')

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Vcch7MG4MgmKlxh0LqUFJzzm', 'function': {'arguments': '{"query":"top rated players Eurocup 2024"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 396, 'total_tokens': 421, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8b8e6789-c16d-45a8-8b18-c847bd748677-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'top rated players Eurocup 2024'}, 'id': 'call_Vcch7MG4MgmKlxh0LqUFJzzm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 396, 'output_tokens': 25, 'total_tokens': 421})]}}
---------------
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.nbcsports.com/soccer/news/euro-2024-player-power-rankings-who-are-the-top-20

**Same question with new thread to see context is not used**

In [24]:
config = {'configurable': {'thread_id': '002'}}
for chunk in agent_executor.stream(
    {'messages': HumanMessage(content='Who where top rated players?')}, config
):
    print(chunk)
    print('---------------')

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jyR73NUROzxrED5nGjnAXZgH', 'function': {'arguments': '{"query":"top rated football players"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 87, 'total_tokens': 108, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-817c8f45-607c-4e7d-9418-2614a5fc5c4d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'top rated football players'}, 'id': 'call_jyR73NUROzxrED5nGjnAXZgH', 'type': 'tool_call'}], usage_metadata={'input_tokens': 87, 'output_tokens': 21, 'total_tokens': 108})]}}
---------------
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.givemesport.com/best-football-players-in-the-world/", "content": "Top 25 Footballers i